## 1. 데이터 로드

In [ ]:
import pandas as pd

# 데이터 불러오기
file_path = './data/10_NEW_fitpet_articles.csv'
df = pd.read_csv(file_path)

# 데이터 확인
print(df.head())
print(df.columns)
df.describe()

## 2. content 전처리

In [1]:
import re
import pandas as pd

# 특정 키워드를 포함하는 문장 제거
def remove_sentences(sentences):
    rm_keywords = ["오시는 길", "이 블로그", "NAVER Corp", "서울특별시", "핏펫", '다음 글']
    clean_sentences = []
    for sentence in sentences:
        if not any(keyword in sentence for keyword in rm_keywords):
            clean_sentences.append(sentence)
    return clean_sentences

def split_by_sentence_end(text):
    # 문장 끝나는 부분(기호 뒤 공백까지) 기준으로 split
    sentences = re.split(r'(?<=[.!?])\s+', text)

    sentences = remove_sentences(sentences)

    # 문장 단위로 나누었다가 다시 합치기 (문장 사이 공백 1개)
    cleaned_text = ' '.join(sentences)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # 혹시 모를 중복 공백 정리
    return cleaned_text.strip()

df['content'] = df['content'].apply(split_by_sentence_end)

def remove_intro(text):
    # '❚'로 시작해서 '메디컬센터'와 '직함'까지 포함하는 패턴을 제거
    # (예: '❚신영식 샤인동물메디컬센터 외과 원장 ')
    pattern = r"^❚\s*.*?장\s+"
    cleaned_text = re.sub(pattern, "", text)

    return cleaned_text

df['content'] = df['content'].apply(remove_intro)

# 3. 특수문자 처리
def remove_special_chars(text):
     # '▶'를 쉼표로 변환
    text = text.replace('▶', ', ')
    text = text.replace('\xa0', ' ')
    text = re.sub(r"[★❚]", "", text)
    return text

df['content'] = df['content'].apply(remove_special_chars)


## 3. title 전처리

In [ ]:
df['title'].head(20)

In [ ]:
def clean_title(title):
    # 1. "- 핏펫" 또는 "- 핏펫 블로그" 제거
    title = re.sub(r'\s*-\s*핏펫( 블로그)?', '', title)

    # 2. 수의사 칼럼 '...' 또는 "..." 패턴 처리
    # 따옴표 안의 내용만 추출
    if "수의사 칼럼" in title:
        match = re.search(r"수의사 칼럼\s*['\"]?(.*?)['\"]?$", title)
        if match:
            title = match.group(1)

    return title

df['title'] = df['title'].apply(clean_title)

## 4. 저장

In [ ]:
# CSV로 저장
df.to_csv('./clean data/10_clean_data.csv', index=False, encoding='utf-8-sig')